In [1]:
import pandas as pd
import numpy as np

from timeit import default_timer as timer

import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

In [21]:
import re

In [2]:
!pwd

/Users/arina/projects/FAERS_graph


In [3]:
raw_data_path = "data/raw/"

In [4]:
!ls data/raw/

ascii_2018q4 xml_2018q4


In [5]:
!ls data/raw/xml_2018q4/xml

1_ADR18Q4.xml 2_ADR18Q4.xml 3_ADR18Q4.xml XML_NTS.pdf   xml18q4.pdf


In [6]:
! head -50 data/raw/xml_2018q4/xml/1_ADR18Q4.xml

<?xml version="1.0"?>
<ichicsr lang="en">
  <ichicsrmessageheader>
    <messagetype>ICSR</messagetype>
    <messageformatversion>2.1</messageformatversion>
    <messageformatrelease>1.0</messageformatrelease>
    <messagenumb>2019-02</messagenumb>
    <messagesenderidentifier>FDA CDER</messagesenderidentifier>
    <messagereceiveridentifier>Public Use</messagereceiveridentifier>
    <messagedateformat>204</messagedateformat>
    <messagedate>20190207040220</messagedate>
  </ichicsrmessageheader>
  <safetyreport>
    <safetyreportversion>1</safetyreportversion>
    <safetyreportid>15529521</safetyreportid>
    <primarysourcecountry>US</primarysourcecountry>
    <occurcountry>US</occurcountry>
    <transmissiondateformat>102</transmissiondateformat>
    <transmissiondate>20190205</transmissiondate>
    <reporttype>1</reporttype>
    <serious>2</serious>
    <receivedateformat>102</receivedateformat>
    <receivedate>20181018</receivedate>
    <receiptdateformat>102</receiptdateformat>
  

In [12]:
import xml.etree.ElementTree as ET

In [13]:
raw_xml_path = "data/raw/xml_2018q4/xml/"

In [14]:
xml_file_1 = raw_xml_path + "1_ADR18Q4.xml"

In [15]:
tree = ET.parse(xml_file_1)
root = tree.getroot()

In [16]:
root.tag

'ichicsr'

In [18]:
root.attrib

{'lang': 'en'}

In [19]:
i=0
for child in root:
    print(child.tag, child.attrib)
    i+=1
    if i>10:
        break

ichicsrmessageheader {}
safetyreport {}
safetyreport {}
safetyreport {}
safetyreport {}
safetyreport {}
safetyreport {}
safetyreport {}
safetyreport {}
safetyreport {}
safetyreport {}


In [20]:
root[0][1].text

'2.1'

In [ ]:
# NOTE: Try using BeautifulSoup instead, b/c I've used it before, it works with xml, and has better support for xpath
# -- No, sticking with ET. I tried BeautifulSoup, and its XML parser took way too long (didn't finish after 15 min).

In [27]:
all_tags = list(set([elem.tag for elem in root.iter()]))

In [28]:
len(all_tags)

87

In [30]:
all_tags.sort()

In [31]:
all_tags

['actiondrug',
 'activesubstance',
 'activesubstancename',
 'authoritynumb',
 'companynumb',
 'drug',
 'drugadditional',
 'drugadministrationroute',
 'drugauthorizationnumb',
 'drugbatchnumb',
 'drugcharacterization',
 'drugcumulativedosagenumb',
 'drugcumulativedosageunit',
 'drugdosageform',
 'drugdosagetext',
 'drugenddate',
 'drugenddateformat',
 'drugindication',
 'drugintervaldosagedefinition',
 'drugintervaldosageunitnumb',
 'drugrecuraction',
 'drugrecurreadministration',
 'drugrecurrence',
 'drugseparatedosagenumb',
 'drugstartdate',
 'drugstartdateformat',
 'drugstructuredosagenumb',
 'drugstructuredosageunit',
 'drugtreatmentduration',
 'drugtreatmentdurationunit',
 'duplicate',
 'duplicatenumb',
 'duplicatesource',
 'fulfillexpeditecriteria',
 'ichicsr',
 'ichicsrmessageheader',
 'literaturereference',
 'medicinalproduct',
 'messagedate',
 'messagedateformat',
 'messageformatrelease',
 'messageformatversion',
 'messagenumb',
 'messagereceiveridentifier',
 'messagesenderiden

In [32]:
i=0
        
for report_id in root.iter('safetyreportid'):
    print(report_id.text)
    i+=1
    if i>10:
        break

15529521
15529522
15529524
15529856
15529858
15529861
15530134
15529556
15529558
15529559
15529564


The ASCIIs look better, let's start with them instead.

In [34]:
!ls data/raw/ascii_2018q4/ascii

ASC_NTS.pdf  INDI18Q4.txt RPSR18Q4.txt drug18q4.pdf reac18q4.pdf
DEMO18Q4.txt OUTC18Q4.txt THER18Q4.txt indi18q4.pdf rpsr18q4.pdf
DRUG18Q4.txt REAC18Q4.txt demo18q4.pdf outc18q4.pdf ther18q4.pdf


In [35]:
!head data/raw/ascii_2018q4/ascii/DEMO18Q4.txt

primaryid$caseid$caseversion$i_f_code$event_dt$mfr_dt$init_fda_dt$fda_dt$rept_cod$auth_num$mfr_num$mfr_sndr$lit_ref$age$age_cod$age_grp$sex$e_sub$wt$wt_cod$rept_dt$to_mfr$occp_cod$reporter_country$occr_country
100035916$10003591$6$F$20130718$20181203$20140312$20181211$EXP$$PHHY2013GB101660$NOVARTIS$$47$YR$$F$Y$$$20181211$$OT$GB$GB
100050413$10005041$3$F$20140306$20141118$20140312$20181213$EXP$$US-TEVA-468475USA$TEVA$$25$YR$$F$Y$68.1$KG$20181213$$CN$US$US
1000551312$10005513$12$F$20120209$20181107$20140313$20181115$EXP$$BR-AMGEN-BRASP2012013548$AMGEN$$55$YR$A$F$Y$67$KG$20181115$$CN$BR$BR
100058832$10005883$2$F$$20180928$20140313$20181012$EXP$$FR-RANBAXY-2014RR-78735$RANBAXY$$31$YR$$F$Y$$$20181012$$OT$GB$FR
100065479$10006547$9$F$201203$20181211$20140313$20181228$EXP$$US-BAYER-2014-035909$BAYER$$36$YR$A$F$Y$90.7$KG$20181228$$CN$US$US
100066188$10006618$8$F$$20181004$20140313$20181017$PER$$US-PFIZER INC-2014069077$PFIZER$$58$YR$$F$Y$$$20181017$$CN$US$US
1000808588$10008085$88$F$20131213$2

In [37]:
raw_ascii_path = "data/raw/ascii_2018q4/ascii/"

In [38]:
ascii_file_demo = raw_ascii_path + "DEMO18Q4.txt"

In [53]:
datatypes = {
    'primaryid': 'object', 
    'caseid': 'object', 
    'caseversion': np.int32, 
    'i_f_code': 'object', 
    'event_dt': 'object', 
    'mfr_dt': 'object',
    'init_fda_dt': 'object', 
    'fda_dt': 'object', 
    'rept_cod': 'object', 
    'auth_num': 'object', 
    'mfr_num': 'object', 
    'mfr_sndr': 'object',
    'lit_ref': 'object', 
    'age': np.int32, 
    'age_cod': 'object', 
    'age_grp': 'object', 
    'sex': 'object', 
    'e_sub': 'object', 
    'wt': np.float64, 
    'wt_cod': 'object',
    'rept_dt': 'object', 
    'to_mfr': 'object', 
    'occp_cod': 'object', 
    'reporter_country': 'object', 
    'occr_country': 'object'
}

# {‘a’: np.float64, ‘b’: np.int32, ‘c’: ‘Int64’}

In [55]:
demo = pd.read_csv(ascii_file_demo, sep='$', dtype='object')

In [56]:
demo.columns

Index(['primaryid', 'caseid', 'caseversion', 'i_f_code', 'event_dt', 'mfr_dt',
       'init_fda_dt', 'fda_dt', 'rept_cod', 'auth_num', 'mfr_num', 'mfr_sndr',
       'lit_ref', 'age', 'age_cod', 'age_grp', 'sex', 'e_sub', 'wt', 'wt_cod',
       'rept_dt', 'to_mfr', 'occp_cod', 'reporter_country', 'occr_country'],
      dtype='object')

In [57]:
demo.wt.head(20)

0       NaN
1      68.1
2        67
3       NaN
4      90.7
5       NaN
6     50.34
7       NaN
8       NaN
9       NaN
10      NaN
11      NaN
12      NaN
13       72
14      NaN
15       79
16      NaN
17      NaN
18      NaN
19      NaN
Name: wt, dtype: object

In [58]:
demo.head()

,primaryid,caseid,caseversion,i_f_code,event_dt,mfr_dt,init_fda_dt,fda_dt,rept_cod,auth_num,...,age_grp,sex,e_sub,wt,wt_cod,rept_dt,to_mfr,occp_cod,reporter_country,occr_country
0,100035916,10003591,6,F,20130718,20181203,20140312,20181211,EXP,NaN,...,NaN,F,Y,NaN,NaN,20181211,NaN,OT,GB,GB
1,100050413,10005041,3,F,20140306,20141118,20140312,20181213,EXP,NaN,...,NaN,F,Y,68.1,KG,20181213,NaN,CN,US,US
2,1000551312,10005513,12,F,20120209,20181107,20140313,20181115,EXP,NaN,...,A,F,Y,67,KG,20181115,NaN,CN,BR,BR
3,100058832,10005883,2,F,NaN,20180928,20140313,20181012,EXP,NaN,...,NaN,F,Y,NaN,NaN,20181012,NaN,OT,GB,FR
4,100065479,10006547,9,F,201203,20181211,20140313,20181228,EXP,NaN,...,A,F,Y,90.7,KG,20181228,NaN,CN,US,US


In [59]:
demo.describe(include='all')

,primaryid,caseid,caseversion,i_f_code,event_dt,mfr_dt,init_fda_dt,fda_dt,rept_cod,auth_num,...,age_grp,sex,e_sub,wt,wt_cod,rept_dt,to_mfr,occp_cod,reporter_country,occr_country
count,394066,394066,394066,394066,205438,370593,394066,394066,394066,20168,...,80189,347760,394066,81142,81142,393749,23479,387070,394066,394053
unique,394066,394066,64,2,4711,2370,2503,183,3,15597,...,6,3,2,3974,2,351,3,5,160,163
top,157739401,15656178,1,I,2018,20181210,20181016,20181016,EXP,0000,...,A,F,Y,70,KG,20181016,N,CN,US,US
freq,1,1,267661,267661,25293,6857,11177,12657,204438,14,...,48200,212580,370587,1404,80809,11312,22175,168973,249968,262062


In [66]:
demo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 394066 entries, 0 to 394065
Data columns (total 25 columns):
primaryid           394066 non-null object
caseid              394066 non-null object
caseversion         394066 non-null object
i_f_code            394066 non-null object
event_dt            205438 non-null object
mfr_dt              370593 non-null object
init_fda_dt         394066 non-null object
fda_dt              394066 non-null object
rept_cod            394066 non-null object
auth_num            20168 non-null object
mfr_num             370595 non-null object
mfr_sndr            394065 non-null object
lit_ref             23441 non-null object
age                 235444 non-null object
age_cod             235452 non-null object
age_grp             80189 non-null object
sex                 347760 non-null object
e_sub               394066 non-null object
wt                  81142 non-null object
wt_cod              81142 non-null object
rept_dt             393749 non-nul

In [61]:
demo.age.value_counts()

70       5562
65       5499
63       5440
60       5394
68       5333
64       5316
62       5312
67       5191
69       5178
61       5177
66       5176
71       5159
59       4903
58       4885
56       4647
57       4605
55       4567
72       4561
75       4526
74       4404
73       4400
54       4260
76       3869
53       3831
52       3748
78       3631
77       3620
50       3488
51       3408
80       3266
         ... 
27533       1
479         1
21658       1
28089       1
23725       1
21723       1
150         1
954         1
28885       1
20244       1
22881       1
21406       1
28888       1
20366       1
754         1
21758       1
22949       1
21472       1
1015        1
14755       1
30448       1
16890       1
26993       1
28742       1
27322       1
16511       1
22021       1
1006        1
1096        1
25109       1
Name: age, Length: 1809, dtype: int64

In [62]:
demo.age.head(50)

0      47
1      25
2      55
3      31
4      36
5      58
6      73
7     NaN
8     NaN
9     NaN
10     47
11     59
12    NaN
13     53
14     27
15     66
16     38
17     69
18    NaN
19     31
20     43
21     22
22     60
23     23
24     82
25     19
26     78
27     50
28     52
29     57
30     34
31     29
32    NaN
33     96
34     53
35     61
36    NaN
37     25
38    NaN
39    NaN
40     69
41     84
42    NaN
43     34
44     37
45     37
46     50
47     73
48     76
49     83
Name: age, dtype: object

In [63]:
demo.age[7]

nan

In [64]:
type(demo.age[7])

float

In [68]:
type(demo.age[6])

str

In [69]:
demo.age[6]

'73'

In [71]:
np.isnan(demo.age[7])

True